In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import skimage as sk
from skimage import measure
import os
import tifffile
from tqdm import tqdm
from typing import List

In [2]:
dots_data = pd.read_csv("/mnt/data/Imaging/202105-Deconwolf/data_210726/dots_data.clean.tsv.gz", sep="\t")

In [3]:
dots_data2 = dots_data.loc["60x" == dots_data["magnification"], :]

In [4]:
matched_dots = pd.read_csv("../../data/60x_reference/matched_dots.global_thr.tsv", sep="\t")

In [5]:
ref_raw = dots_data2.loc["raw" == dots_data2["image_type"], :].reset_index(drop=True)
ref__dw = dots_data2.loc["dw" == dots_data2["image_type"], :].reset_index(drop=True)
field_list = np.unique(dots_data2["sid"].values).tolist()
raw_thr = 0.15
dw__thr = 5.3

In [6]:
all_raw_dots: List[pd.DataFrame] = []
all_dw__dots: List[pd.DataFrame] = []
selected_raw_dots: List[pd.DataFrame] = []
selected_dw__dots: List[pd.DataFrame] = []
for current_field_id in field_list:
    print(f"Field #{current_field_id}")
    field_raw_dots = ref_raw.loc[ref_raw["sid"] == current_field_id, :].sort_values("Value2", ascending=False)
    field_dw__dots = ref__dw.loc[ref__dw["sid"] == current_field_id, :].sort_values("Value2", ascending=False)
    
    matched_raw_dots = matched_dots.loc[matched_dots["series"] == current_field_id, "raw_id"].values
    matched_dw__dots = matched_dots.loc[matched_dots["series"] == current_field_id, "dw__id"].values
    
    all_raw_dots.append(field_raw_dots.loc[field_raw_dots["Value2"] >= raw_thr, :].reset_index(drop=True))
    all_dw__dots.append(field_dw__dots.loc[field_dw__dots["Value2"] >= dw__thr, :].reset_index(drop=True))
    selected_raw_dots.append(field_raw_dots.loc[field_raw_dots["Value2"] >= raw_thr, :].reset_index(
        drop=True).iloc[matched_raw_dots, :])
    selected_dw__dots.append(field_dw__dots.loc[field_dw__dots["Value2"] >= dw__thr, :].reset_index(
        drop=True).iloc[matched_dw__dots, :])

Field #1
Field #2
Field #3
Field #4
Field #5
Field #6
Field #7


In [7]:
pd.concat(all_raw_dots).reset_index(drop=True).to_csv("../../data/60x_reference/raw.global_thr.tsv", sep="\t", index=False)
pd.concat(selected_raw_dots).reset_index(drop=True).to_csv("../../data/60x_reference/ref_raw.global_thr.tsv", sep="\t", index=False)

In [8]:
pd.concat(all_dw__dots).reset_index(drop=True).to_csv("../../data/60x_reference/dw.global_thr.tsv", sep="\t", index=False)
pd.concat(selected_dw__dots).reset_index(drop=True).to_csv("../../data/60x_reference/ref__dw.global_thr.tsv", sep="\t", index=False)